## Meteo forcing example

This is an example of reading locally stored ECMWF grib files. 

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.meteo as pmeteo
import datetime
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import pandas as pd

In [ ]:
from hvplot import pandas,xarray

In [ ]:
pyPoseidon.__version__

### setup

In [ ]:
#define in a dictionary the properties of the model..
dic={'minlon':-35., # lat/lon window
     'maxlon':42.,
     'minlat':25.05,
     'maxlat':76.5,
     'start_date':'2010-2-1',
     'end_date':'2010-2-2'
    }

In [ ]:
# This just a structure to create the list of paths for the actual meteo files - change appropriately 
dr = pd.date_range(dic['start_date'],dic['end_date'], freq='12H')
#creating a sequence of folder from which we read the meteo.
#PATH='/Volumes/data/projects/CRITECH/meteo/ECMWF/operational/'
PATH='/Users/brey/DATA/'#Path to meteo files
folders = [datetime.datetime.strftime(x, '%Y%m%d.%H') for x in dr]
meteo = [PATH+'{:04d}/{:02d}/{:02d}/'.format(x.year,x.month,x.day)+datetime.datetime.strftime(x, '%Y%m%d.%H')+'.tropical_cyclone.grib' for x in dr]
meteo

In [ ]:
dic.update({'mpaths':meteo,'ft1':0,'ft2':3})

### retrieve UVP

In [ ]:
dic.update(engine='pynio')

In [ ]:
#get uvp
meteo = pmeteo.combine_meteo(**dic)

In [ ]:
meteo.uvp

In [ ]:
meteo.uvp.time

## Visualise

### holoviews

In [ ]:
meteo.uvp.msl.hvplot.contourf(x='longitude',y='latitude')

### geoviews

In [ ]:
g50 = gf.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
meteo.uvp.msl.hvplot.quadmesh(x='longitude',y='latitude',geo=True, rasterize=True) * g50 * gf.borders()

## Output to file

In [ ]:
#generic netCDF output
meteo.uvp.to_netcdf('./test/test.nc') # to netcdf

In [ ]:
#save to DELFT3D format
meteo.output(solver='d3d',rpath='./test/') # to u,v,p for d3d

In [ ]:
#save to SCHISM format
meteo.output(solver='schism',rpath='./test/')